<a href="https://colab.research.google.com/github/haidour18/NLP/blob/main/Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
#install dependecies
!pip install kaggle
!pip install -q keras
!pip install numpy
!pip install pandas
!pip install Attention 
!pip install keras-self-attention
!pip install keras-attention


In [3]:
from google.colab import  drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from google.colab import files 
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"asmaahaidour","key":"9558ae9a1dcd81c5213ff117d35c9658"}'}

In [6]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
#change permissions 
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets download -d hsankesara/medium-articles

  0% 0.00/1.34M [00:00<?, ?B/s]
100% 1.34M/1.34M [00:00<00:00, 93.2MB/s]


Done


In [12]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os, glob , csv
import time, joblib
import pandas as pd
#from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import seaborn as sns

In [14]:
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords

In [18]:
import numpy as np
import pandas as pd 
import re
from keras.preprocessing.text import Tokenizer 
from nltk import download
download('stopwords')
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
from keras_self_attention import SeqSelfAttention
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
#Let’s see some top 5 headline and text, and print our data shape.

reviews = pd.read_csv("articles.csv")
print(reviews.shape)
reviews.head()

(337, 6)


,author,claps,reading_time,link,title,text
0,Justin Lee,8.3K,11,https://medium.com/swlh/chatbots-were-the-next-big-thing-what-happened-5fc49dd6fa61?source=---------0----------------,Chatbots were the next big thing: what happened? – The Startup – Medium,"Oh, how the headlines blared:\nChatbots were The Next Big Thing.\nOur hopes were sky high. Bright-eyed and bushy-tailed, the industry was ripe for a new era of innovation: it was time to start soc..."
1,Conor Dewey,1.4K,7,https://towardsdatascience.com/python-for-data-science-8-concepts-you-may-have-forgotten-i-did-825966908393?source=---------1----------------,Python for Data Science: 8 Concepts You May Have Forgotten,"If you’ve ever found yourself looking up the same question, concept, or syntax over and over again when programming, you’re not alone.\nI find myself doing this constantly.\nWhile it’s not unnatur..."
2,William Koehrsen,2.8K,11,https://towardsdatascience.com/automated-feature-engineering-in-python-99baf11cc219?source=---------2----------------,Automated Feature Engineering in Python – Towards Data Science,"Machine learning is increasingly moving from hand-designed models to automatically optimized pipelines using tools such as H20, TPOT, and auto-sklearn. These libraries, along with methods such as ..."
3,Gant Laborde,1.3K,7,https://medium.freecodecamp.org/machine-learning-how-to-go-from-zero-to-hero-40e26f8aa6da?source=---------3----------------,Machine Learning: how to go from Zero to Hero – freeCodeCamp,"If your understanding of A.I. and Machine Learning is a big question mark, then this is the blog post for you. Here, I gradually increase your AwesomenessicityTM by gluing inspirational videos tog..."
4,Emmanuel Ameisen,935,11,https://blog.insightdatascience.com/reinforcement-learning-from-scratch-819b65f074d8?source=---------4----------------,Reinforcement Learning from scratch – Insight Data,"Want to learn about applied Artificial Intelligence from leading practitioners in Silicon Valley, New York, or Toronto? Learn more about the Insight Artificial Intelligence Fellows Program.\nAre y..."


In [20]:
#Check Null values 
reviews.isnull().sum()

author          0
claps           0
reading_time    0
link            0
title           0
text            0
dtype: int64